In [ ]:
import re
import json
import csv
import os
from collections import defaultdict, Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy
from ebooklib import epub
from bs4 import BeautifulSoup
import copy


import os
import nltk

# -------------------------- 核心修改：修正NLTK数据路径 --------------------------
# 新路径（正确）：改为实际的AppData路径
NLTK_DATA_DIR = "C:\\Users\\UTU\\AppData\\Roaming\\nltk_data"

# 后续的路径配置同步使用这个正确的根路径
os.environ["NLTK_DATA"] = NLTK_DATA_DIR
nltk.data.path = [NLTK_DATA_DIR]
print(f"✅ NLTK数据集路径已指定为：{NLTK_DATA_DIR}")

# 验证数据集的路径也会自动基于正确的根路径拼接
required_datasets = {
    "stopwords": os.path.join(NLTK_DATA_DIR, "corpora", "stopwords", "english"),
    "wordnet": os.path.join(NLTK_DATA_DIR, "corpora", "wordnet"),
    "punkt": os.path.join(NLTK_DATA_DIR, "tokenizers", "punkt", "PY3", "english.pickle"),
    "averaged_perceptron_tagger": os.path.join(NLTK_DATA_DIR, "taggers", "averaged_perceptron_tagger", "averaged_perceptron_tagger.pickle")
}

# 检查数据集是否存在
for data_name, data_path in required_datasets.items():
    if not os.path.exists(data_path):
        raise FileNotFoundError(
            f"❌ 未找到{NLTK_DATA_DIR}下的{data_name}数据集！\n"
            f"请确认{data_path}文件/文件夹存在"
        )
print("✅ 所有NLTK数据集均存在！")

# -------------------------- 初始化NLTK核心依赖（直接加载手动数据集） --------------------------
# 停用词（英文）
STOP_WORDS = set(stopwords.words('english'))
# 词形还原器
lemmatizer = WordNetLemmatizer()
# 验证词性标注器（确保手动数据集可用）
try:
    nltk.pos_tag(word_tokenize("test sentence"))
except Exception as e:
    raise Exception(f"❌ 词性标注数据集加载失败！请检查：\n{NLTK_DATA_DIR}\\taggers\\averaged_perceptron_tagger\\averaged_perceptron_tagger.pickle")

# -------------------------- 初始化spaCy（确保nlp可调用，适配命名实体提取） --------------------------
try:
    nlp = spacy.load("en_core_web_sm")
    print("✅ spaCy模型en_core_web_sm加载成功")
except OSError:
    raise Exception("❌ spaCy模型未安装！请先运行：python -m spacy download en_core_web_sm")
except Exception as e:
    raise Exception(f"❌ spaCy初始化失败：{str(e)}")

# -------------------------- 可配置参数（适配课程要求，无需修改路径） --------------------------
INPUT_EPUB = "./Artificial Intelligence_ A Modern Approach 4th Ed.epub"  # 你的英文版EPUB路径
TARGET_CHAPTERS = [1, 2, 3, 4, 5]  # 目标章节（1-5章）
MIN_TEXT_LENGTH = 20000  # 文本规模≥2万字
# 输出文件（支撑多智能体工作流+人机协作）
PREPROCESS_JSON = "preprocess_result.json"
TERM_TABLE = "terminology_verification.csv"
NER_TABLE = "named_entity_verification.csv"
STYLE_METADATA = "style_analysis.csv"
CULTURAL_WORD_TABLE = "cultural_word_verification.csv"
TRANSLATION_UNITS = "translation_units.csv"

import re
from collections import defaultdict
from ebooklib import epub
from bs4 import BeautifulSoup

# -------------------------- 辅助函数：数字转罗马数字 --------------------------
def roman_numeral(n):
    val = [1000, 900, 500, 400, 100, 90, 50, 40, 10, 9, 5, 4, 1]
    syb = ["M", "CM", "D", "CD", "C", "XC", "L", "XL", "X", "IX", "V", "IV", "I"]
    roman_num = ''
    i = 0
    while n > 0:
        for _ in range(n // val[i]):
            roman_num += syb[i]
            n -= val[i]
        i += 1
    return roman_num

# -------------------------- 核心功能1：EPUB章节提取--------------------------
def extract_epub_chapters(epub_path, target_chapters, MIN_TEXT_LENGTH=20000):
    # 1. 读取EPUB文件
    book = epub.read_epub(epub_path)
    all_items = []  # 存储EPUB的每个文本项（标题+内容）

    # 2. 遍历EPUB的所有项，提取文本内容（关键：填充all_items）
    for item in book.get_items():
        # 只处理HTML/XHTML文本项
        if item.media_type in ["application/xhtml+xml", "text/html"]:
            try:
                content = item.get_content().decode('utf-8')
            except UnicodeDecodeError:
                # 兼容特殊编码
                content = item.get_content().decode('latin-1', errors='replace')
            
            # 解析HTML，提取标题和正文
            soup = BeautifulSoup(content, 'html.parser')
            # 提取章节标题（h1/h2/h3标签）
            title_tag = soup.find(['h1', 'h2', 'h3'])
            section_title = title_tag.get_text(strip=True) if title_tag else ""
            
            # 清理正文（移除无关标签，保留文本结构）
            for elem in soup(["script", "style", "img", "svg", "nav", "footer", "header", "table"]):
                elem.decompose()
            text = soup.get_text(separator='\n', strip=True)
            # 只合并4个以上的换行，保留章节标题的换行（适配你的格式）
            text = re.sub(r'\n{4,}', '\n\n\n', text)
            
            # 将项添加到all_items
            all_items.append({
                "title": section_title,
                "content": text,
                "id": item.get_id(),
                "order": len(all_items)
            })

    # 3. 拼接完整文本（现在all_items有内容了）
    full_text = "\n".join([f"{item['title']}\n{item['content']}" for item in all_items])
    # 再次清理换行，确保章节标题独立成行
    full_text = re.sub(r'\n{4,}', '\n\n\n', full_text)

    # 4. 校验总文本长度（避免文本过短）
    total_len = len(full_text.replace(' ', ''))
    print(f"📚 提取英文版总文本字符数：{total_len}（需≥{MIN_TEXT_LENGTH}）")
    if total_len < MIN_TEXT_LENGTH:
        raise ValueError(f"文本规模不达标！当前{total_len}字符，需≥{MIN_TEXT_LENGTH}字符")

    # 5. 初始化chapters_data（关键：解决未定义问题）
    chapters_data = defaultdict(dict)  # 用defaultdict更方便，也可以用普通字典：chapters_data = {}

    # 6. 提取目标章节（适配你的CHAPTER1格式）
    for chap_num in target_chapters:
        chap_content = ""
        chap_title = f"Chapter {chap_num}"
        
        # -------------------------- 适配你的格式：匹配“CHAPTER1”“CHAPTER2” --------------------------
        patterns = [
            rf'(?i)CHAPTER{chap_num}\n([^\n]*)\n([\s\S]*?)(?=(?i)CHAPTER{chap_num+1}|$)'
            # 解释：
            # (?i)：忽略大小写（适配chapter1/CHAPTER1）
            # CHAPTER{chap_num}：匹配CHAPTER1/CHAPTER2（无空格）
            # \n([^\n]*)\n：捕获副标题（如INTRODUCTION）
            # ([\s\S]*?)：非贪婪匹配章节内容
            # (?=CHAPTER{chap_num+1}|$)：直到下一章或文本结束
        ]
        
        # 尝试正则匹配
        for pattern in patterns:
            match = re.search(pattern, full_text)
            if match:
                # 提取副标题
                sub_title = match.group(1).strip()
                chap_title = f"Chapter {chap_num}: {sub_title}"
                # 提取章节内容
                chap_content = match.group(2).strip()
                print(f"✅ 成功匹配章节{chap_num}：标题={chap_title}")
                break
        
        # -------------------------- 兜底分割：强制章节1/2不重叠 --------------------------
        chap_content_clean = chap_content.replace(' ', '')
        if len(chap_content_clean) < 10000:
            print(f"⚠️  章节{chap_num}匹配失败，强制按章节标题位置分割")
            # 找到CHAPTER1/CHAPTER2的位置
            chap1_pos = full_text.find("CHAPTER1")
            chap2_pos = full_text.find("CHAPTER2")
            if chap_num == 1:
                # 章节1：从CHAPTER1到CHAPTER2
                chap_content = full_text[chap1_pos:chap2_pos].strip()
            else:
                # 章节2：从CHAPTER2到文本结束
                chap_content = full_text[chap2_pos:].strip()
        
        # -------------------------- 分割逻辑段落（需确保split_into_logical_paragraphs函数存在） --------------------------
        paragraphs = split_into_logical_paragraphs(chap_content, chap_num)
        
        # -------------------------- 填充章节数据 --------------------------
        chapters_data[chap_num] = {
            "title": chap_title,
            "raw_content": chap_content,
            "paragraphs": paragraphs,
            "para_count": len(paragraphs),
            "char_count": len(chap_content.replace(' ', ''))
        }
        print(f"✅ 章节{chap_num}：{chapters_data[chap_num]['para_count']}个逻辑段落，{chapters_data[chap_num]['char_count']}字符")

    # 7. 返回章节数据
    return chapters_data

# 需确保split_into_logical_paragraphs函数存在（之前的代码里的函数）
def split_into_logical_paragraphs(text, chap_num):
    raw_paragraphs = re.split(r'(?<=[.!?])\s*\n+', text)
    logical_paras = []
    for idx, para in enumerate(raw_paragraphs):
        para = para.strip()
        if not para:
            continue
        prev_para_id = f"chap{chap_num}_para{idx}" if idx > 0 else "None"
        next_para_id = f"chap{chap_num}_para{idx+2}" if idx < len(raw_paragraphs)-2 else "None"
        para_id = f"chap{chap_num}_para{idx+1}"
        sentences = nltk.sent_tokenize(para)
        logical_paras.append({
            "para_id": para_id,
            "content": para,
            "sentence_count": len(sentences),
            "prev_para_id": prev_para_id,
            "next_para_id": next_para_id,
            "char_count": len(para.replace(' ', ''))
        })
    return logical_paras

# -------------------------- 核心功能2：语域与风格自动识别 --------------------------
def analyze_style_and_domain(para_content, para_id):
    ai_domain_keywords = {
        "machine learning", "neural network", "reinforcement learning", "bayesian network",
        "convolutional neural network", "attention mechanism", "backpropagation",
        "markov decision process", "perceptron", "rational agent"
    }
    formal_style_keywords = {"furthermore", "nevertheless", "consequently", "therefore", "moreover", "however"}
    
    # 基础统计（使用手动加载的tokenize/sent_tokenize）
    sentences = sent_tokenize(para_content)
    sent_count = len(sentences)
    words = word_tokenize(para_content.lower())
    word_count = len(words)
    avg_sent_length = round(word_count / sent_count, 2) if sent_count > 0 else 0
    
    # 被动语态占比
    passive_count = len(re.findall(r'\bis\s+\w+ed\b|\bwas\s+\w+ed\b|\bwere\s+\w+ed\b', para_content.lower()))
    passive_ratio = round(passive_count / word_count, 3) if word_count > 0 else 0
    
    # 领域与风格判断
    text_word_set = set(words)
    domain_match = text_word_set & ai_domain_keywords
    domain = "AI_Professional" if len(domain_match) >= 3 else "AI_Popular"
    
    formal_match = text_word_set & formal_style_keywords
    formality_score = 7.0 + len(formal_match)*0.5 + (1 if passive_ratio >= 0.05 else 0) + (1 if avg_sent_length >= 25 else 0)
    formality_score = min(formality_score, 10.0)
    style = "Formal_Academic" if formality_score >= 8.0 else "Semi_Formal_Academic"
    
    return {
        "para_id": para_id,
        "domain": domain,
        "style": style,
        "formality_score": round(formality_score, 1),
        "avg_sentence_length": avg_sent_length,
        "passive_voice_ratio": passive_ratio,
        "domain_keywords": list(domain_match),
        "formal_keywords": list(formal_match),
        "word_count": word_count,
        "sentence_count": sent_count
    }

# -------------------------- 核心功能3：领域术语提取（直接使用手动词性标注数据集） --------------------------
def extract_terminology(para_content):
    # 分词与过滤（使用手动加载的stopwords）
    tokens = word_tokenize(para_content.lower())
    filtered_tokens = [
        token for token in tokens 
        if token not in STOP_WORDS and not token.isdigit() and len(token) >= 3
    ]
    
    # 词性标注（使用手动加载的averaged_perceptron_tagger）
    pos_tags = nltk.pos_tag(filtered_tokens)
    valid_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS']
    valid_tokens = [lemmatizer.lemmatize(token) for token, pos in pos_tags if pos in valid_pos_tags]
    
    # 补充AI核心术语
    ai_core_terms = {
        "rational agent", "bayesian network", "machine learning", "neural network",
        "reinforcement learning", "markov decision process", "perceptron",
        "attention mechanism", "backpropagation", "convolutional neural network"
    }
    for term in ai_core_terms:
        valid_tokens.append(term)
    
    return Counter(valid_tokens)

# -------------------------- 核心功能4：命名实体提取（使用手动加载的spaCy） --------------------------
def extract_named_entities(para_content, para_id):
    doc = nlp(para_content)
    named_entities = []
    target_entity_types = ["PERSON", "ORG", "GPE", "WORK_OF_ART", "PRODUCT", "EVENT"]
    
    for ent in doc.ents:
        if ent.label_ in target_entity_types:
            context_start = max(0, ent.start_char - 50)
            context_end = min(len(para_content), ent.end_char + 50)
            entity_context = para_content[context_start:context_end]
            
            named_entities.append({
                "para_id": para_id,
                "entity_text": ent.text,
                "entity_type": ent.label_,
                "start_char": ent.start_char,
                "end_char": ent.end_char,
                "context": entity_context
            })
    return named_entities

# -------------------------- 核心功能5：文化负载词提取 --------------------------
def extract_cultural_words(para_content, para_id):
    cultural_word_lib = {
        "state-of-the-art": {"semantic_explanation": "AI领域指当前最高技术水平", "translation_strategy": "意译‘前沿技术’"},
        "holy grail": {"semantic_explanation": "比喻领域终极目标", "translation_strategy": "意译‘圣杯（终极目标）’"},
        "trial and error": {"semantic_explanation": "AI训练中的试错方法", "translation_strategy": "固定译法‘试错法’"},
        "game-changer": {"semantic_explanation": "颠覆性技术/方法", "translation_strategy": "意译‘变革性技术’"}
    }
    
    extracted = []
    for word, info in cultural_word_lib.items():
        if re.search(r'\b' + re.escape(word) + r'\b', para_content, re.IGNORECASE):
            match = re.search(r'\b' + re.escape(word) + r'\b', para_content, re.IGNORECASE)
            context_start = max(0, match.start() - 50)
            context_end = min(len(para_content), match.end() + 50)
            
            extracted.append({
                "para_id": para_id,
                "cultural_word": word,
                "semantic_explanation": info["semantic_explanation"],
                "translation_strategy": info["translation_strategy"],
                "context": para_content[context_start:context_end]
            })
    return extracted




# -------------------------- 新增：递归转换所有set为list（彻底解决JSON序列化） --------------------------
# 递归转换set为list的函数
def convert_set_to_list(obj):
    if isinstance(obj, dict):
        return {key: convert_set_to_list(value) for key, value in obj.items()}
    elif isinstance(obj, list) or isinstance(obj, tuple):
        return [convert_set_to_list(item) for item in obj]
    elif isinstance(obj, set):
        return list(obj)
    else:
        return obj


# -------------------------- 核心功能6：保存预处理成果（用于后续人机协作） --------------------------
def save_preprocess_results(final_data):
    data_to_save = copy.deepcopy(final_data)
    data_to_save = convert_set_to_list(data_to_save)  # 递归转换所有set
    
    # -------------------------- 原保存逻辑不变（使用转换后的数据） --------------------------
    # 1. 完整结构化数据（供多智能体调用）
    with open(PREPROCESS_JSON, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=2)  # 用转换后的数据
    print(f"✅ 完整数据已保存至：{PREPROCESS_JSON}")

    # 2. 术语校验表（人机协作）
    with open(TERM_TABLE, 'w', encoding='utf-8', newline='') as f:
        fieldnames = ["术语", "出现次数", "领域分类", "建议译法（人工填写）", "校验结果（通过/修改）"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        ai_core_terms = final_data["ai_core_terms"]  # 原数据不影响CSV，无需转换
        for term, count in sorted(final_data["global_terminology"].items(), key=lambda x: -x[1]):
            domain = "AI专业术语" if term in ai_core_terms else "通用术语"
            writer.writerow({
                "术语": term, "出现次数": count, "领域分类": domain,
                "建议译法（人工填写）": "", "校验结果（通过/修改）": ""
            })
    print(f"✅ 术语校验表已保存至：{TERM_TABLE}")

    # 3. 命名实体校验表（人机协作）
    all_entities = []
    entity_counter = Counter()
    for chap in final_data["chapters"]:
        for para in chap["paragraphs"]:
            for ent in para["named_entities"]:
                all_entities.append(ent)
                entity_counter[(ent["entity_text"], ent["entity_type"])] += 1
    # 去重并添加出现次数
    unique_entities = []
    seen = set()
    for ent in all_entities:
        key = (ent["entity_text"], ent["entity_type"])
        if key not in seen:
            seen.add(key)
            ent["出现次数"] = entity_counter[key]
            unique_entities.append(ent)
    with open(NER_TABLE, 'w', encoding='utf-8', newline='') as f:
        fieldnames = ["实体原文", "实体类型", "出现次数", "建议译法（人工填写）", "校验结果（通过/修改）", "上下文"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for ent in unique_entities:
            writer.writerow({
                "实体原文": ent["entity_text"], "实体类型": ent["entity_type"], "出现次数": ent["出现次数"],
                "建议译法（人工填写）": "", "校验结果（通过/修改）": "", "上下文": ent["context"]
            })
    print(f"✅ 命名实体校验表已保存至：{NER_TABLE}")

    # 4. 风格分析表（指导翻译Agent）
    style_data = []
    for chap in final_data["chapters"]:
        for para in chap["paragraphs"]:
            style_meta = para["style_metadata"]
            style_data.append({
                "段落ID": style_meta["para_id"], "章节号": chap["chapter_num"],
                "领域": style_meta["domain"], "语体风格": style_meta["style"],
                "正式度评分": style_meta["formality_score"], "平均句长": style_meta["avg_sentence_length"],
                "被动语态占比": style_meta["passive_voice_ratio"], "领域关键词": ",".join(style_meta["domain_keywords"])
            })
    with open(STYLE_METADATA, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=style_data[0].keys())
        writer.writeheader()
        writer.writerows(style_data)
    print(f"✅ 风格分析表已保存至：{STYLE_METADATA}")

    # 5. 文化负载词表（人机协作）
    all_cultural = []
    for chap in final_data["chapters"]:
        for para in chap["paragraphs"]:
            all_cultural.extend(para["cultural_words"])
    # 原代码中保存文化负载词表的部分
    with open(CULTURAL_WORD_TABLE, 'w', encoding='utf-8', newline='') as f:
        fieldnames = ["文化负载词", "语义解释", "翻译策略", "上下文", "最终译法（人工填写）", "校验结果"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        # 原错误代码：writer.writerows(all_cultural)
        # 新代码：手动映射英文键到中文列名
        for item in all_cultural:
            # 生成符合fieldnames的新字典
            csv_row = {
                "文化负载词": item["cultural_word"],
                "语义解释": item["semantic_explanation"],
                "翻译策略": item["translation_strategy"],
                "上下文": item["context"],
                "最终译法（人工填写）": "",  # 空值，供人工填写
                "校验结果": ""  # 空值，供人工填写
            }
            writer.writerow(csv_row)  # 写入单个行
    print(f"✅ 文化负载词表已保存至：{CULTURAL_WORD_TABLE}")

    # 6. 翻译单元表（供翻译Agent执行）
    translation_units = []
    for chap in final_data["chapters"]:
        for para in chap["paragraphs"]:
            para_terms = [t for t, _ in extract_terminology(para["content"]).most_common(3)]
            para_ents = [ent["entity_text"] for ent in para["named_entities"]]
            translation_units.append({
                "单元ID": para["para_id"], "章节号": chap["chapter_num"],
                "原文": para["content"], "上下文关联": f"前：{para['prev_para_id']} | 后：{para['next_para_id']}",
                "领域": para["style_metadata"]["domain"], "语体要求": para["style_metadata"]["style"],
                "关键术语": ",".join(para_terms), "命名实体": ",".join(para_ents) if para_ents else "无",
                "Agent译文": "", "人工修正": ""
            })
    with open(TRANSLATION_UNITS, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=translation_units[0].keys())
        writer.writeheader()
        writer.writerows(translation_units)
    print(f"✅ 翻译单元表已保存至：{TRANSLATION_UNITS}")

# -------------------------- 主函数（整合所有预处理流程） --------------------------
def main():
    print(f"步骤1/6：解析英文版EPUB {INPUT_EPUB}...")
    # 提取章节（直接使用手动NLTK）
    chapters_data = extract_epub_chapters(INPUT_EPUB, TARGET_CHAPTERS)
    
    # 初始化全局数据结构
    final_data = {
        "book_info": "Artificial Intelligence: A Modern Approach (4th Ed, English)",
        "target_chapters": TARGET_CHAPTERS,
        "global_metadata": {
            "total_characters": 0, "total_paragraphs": 0, "total_terms": 0,
            "total_entities": 0, "total_cultural_words": 0,
            "dominant_domain": "AI_Professional", "dominant_style": "Formal_Academic"
        },
        "chapters": [],
        "global_terminology": {},
        "ai_core_terms": {"rational agent", "bayesian network", "machine learning", "neural network",
                          "reinforcement learning", "markov decision process", "perceptron",
                          "attention mechanism", "backpropagation", "convolutional neural network"}
    }
    
    # 处理每个章节
    global_terms = Counter()
    global_entity_count = 0
    global_cultural_count = 0
    
    for chap_num, chap_data in chapters_data.items():
        print(f"步骤2/6：处理章节{chap_num}（{chap_data['char_count']}字符）...")
        processed_paras = []
        chap_content = chap_data["raw_content"]
        
        for para in chap_data["paragraphs"]:
            para_id = para["para_id"]
            para_text = para["content"]
            
            # 提取多维度信息（直接使用手动加载的依赖）
            style_meta = analyze_style_and_domain(para_text, para_id)
            para_terms = extract_terminology(para_text)
            para_ents = extract_named_entities(para_text, para_id)
            para_cultural = extract_cultural_words(para_text, para_id)
            
            # 汇总全局数据
            global_terms.update(para_terms)
            global_entity_count += len(para_ents)
            global_cultural_count += len(para_cultural)
            
            processed_paras.append({
                "para_id": para_id, "content": para_text,
                "sentence_count": para["sentence_count"], "char_count": para["char_count"],
                "prev_para_id": para["prev_para_id"], "next_para_id": para["next_para_id"],
                "style_metadata": style_meta, "terminology": dict(para_terms),
                "named_entities": para_ents, "cultural_words": para_cultural
            })
        
        # 保存章节数据
        final_data["chapters"].append({
            "chapter_num": chap_num, "title": chap_data["title"],
            "paragraphs": processed_paras, "para_count": len(processed_paras),
            "char_count": chap_data["char_count"]
        })
        final_data["global_metadata"]["total_characters"] += chap_data["char_count"]
        final_data["global_metadata"]["total_paragraphs"] += len(processed_paras)
    
    # 填充全局元数据
    final_data["global_terminology"] = dict(global_terms)
    final_data["global_metadata"]["total_terms"] = len(global_terms)
    final_data["global_metadata"]["total_entities"] = global_entity_count
    final_data["global_metadata"]["total_cultural_words"] = global_cultural_count
    
    # 保存成果
    print(f"步骤3/6：保存预处理成果...")
    save_preprocess_results(final_data)
    
    # 输出统计
    print(f"\n🎉 预处理完成！全局统计：")
    print(f"  - 目标章节：{TARGET_CHAPTERS}")
    print(f"  - 总字符数：{final_data['global_metadata']['total_characters']}（≥{MIN_TEXT_LENGTH}）")
    print(f"  - 翻译单元数：{final_data['global_metadata']['total_paragraphs']}")
    print(f"  - 提取术语数：{final_data['global_metadata']['total_terms']}")
    print(f"  - 命名实体数：{final_data['global_metadata']['total_entities']}")

if __name__ == "__main__":
    main()

✅ NLTK数据集路径已指定为：C:\Users\UTU\AppData\Roaming\nltk_data
✅ 所有NLTK数据集均存在！
✅ spaCy模型en_core_web_sm加载成功
步骤1/6：解析英文版EPUB ./Artificial Intelligence_ A Modern Approach 4th Ed.epub...
📚 提取英文版总文本字符数：3398053（需≥20000）
✅ 成功匹配章节1：标题=Chapter 1: CHAPTER
✅ 章节1：264个逻辑段落，105690字符
✅ 成功匹配章节2：标题=Chapter 2: CHAPTER
✅ 章节2：204个逻辑段落，73487字符
✅ 成功匹配章节3：标题=Chapter 3: CHAPTER
✅ 章节3：417个逻辑段落，129519字符
✅ 成功匹配章节4：标题=Chapter 4: CHAPTER
✅ 章节4：278个逻辑段落，95372字符
✅ 成功匹配章节5：标题=Chapter 5: CHAPTER


C:\Users\UTU\AppData\Local\Temp\ipykernel_35124\3646716385.py:165: DeprecationWarning: Flags not at the start of the expression '(?i)CHAPTER1\\n([^\\n]' (truncated) but at position 37
  match = re.search(pattern, full_text)
C:\Users\UTU\AppData\Local\Temp\ipykernel_35124\3646716385.py:165: DeprecationWarning: Flags not at the start of the expression '(?i)CHAPTER2\\n([^\\n]' (truncated) but at position 37
  match = re.search(pattern, full_text)
C:\Users\UTU\AppData\Local\Temp\ipykernel_35124\3646716385.py:165: DeprecationWarning: Flags not at the start of the expression '(?i)CHAPTER3\\n([^\\n]' (truncated) but at position 37
  match = re.search(pattern, full_text)
C:\Users\UTU\AppData\Local\Temp\ipykernel_35124\3646716385.py:165: DeprecationWarning: Flags not at the start of the expression '(?i)CHAPTER4\\n([^\\n]' (truncated) but at position 37
  match = re.search(pattern, full_text)
C:\Users\UTU\AppData\Local\Temp\ipykernel_35124\3646716385.py:165: DeprecationWarning: Flags not at the 

✅ 章节5：231个逻辑段落，81672字符
步骤2/6：处理章节1（105690字符）...
步骤2/6：处理章节2（73487字符）...
步骤2/6：处理章节3（129519字符）...
步骤2/6：处理章节4（95372字符）...
步骤2/6：处理章节5（81672字符）...
步骤3/6：保存预处理成果...
✅ 完整数据已保存至：preprocess_result.json
✅ 术语校验表已保存至：terminology_verification.csv
✅ 命名实体校验表已保存至：named_entity_verification.csv
✅ 风格分析表已保存至：style_analysis.csv
✅ 文化负载词表已保存至：cultural_word_verification.csv
✅ 翻译单元表已保存至：translation_units.csv

🎉 预处理完成！全局统计：
  - 目标章节：[1, 2, 3, 4, 5]
  - 总字符数：485740（≥20000）
  - 翻译单元数：1394
  - 提取术语数：6049
  - 命名实体数：2336


In [4]:
import re
import json
import pandas as pd
from typing import List, Dict, Optional, Tuple
import hashlib
from pathlib import Path
import csv
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class MarkdownToElasticsearchConverter:
    """将Markdown表格转换为Elasticsearch可读取格式"""
    
    def __init__(self, source_lang: str = 'en', target_lang: str = 'zh'):
        """
        初始化转换器
        
        Args:
            source_lang: 源语言代码，默认为'en'
            target_lang: 目标语言代码，默认为'zh'
        """
        self.source_lang = source_lang
        self.target_lang = target_lang
        
    def parse_markdown_table(self, markdown_content: str) -> List[Dict]:
        """
        解析Markdown表格内容（修复空列导致的列数不匹配问题）
        
        Args:
            markdown_content: Markdown内容字符串
            
        Returns:
            解析后的字典列表
        """
        lines = markdown_content.strip().split('\n')
        if not lines:
            logger.warning("Markdown内容为空")
            return []
        
        # 找到表格开始和结束位置（优化：避免空行导致提前终止）
        table_lines = []
        in_table = False
        
        for line in lines:
            stripped_line = line.strip()
            if '|' in stripped_line:
                in_table = True
                table_lines.append(stripped_line)
            # 仅当非空行且不含|时，才终止表格解析
            elif in_table and stripped_line:
                break
        
        if len(table_lines) < 2:
            logger.warning("表格行数不足（至少需要表头+分隔行）")
            return []
        
        # 解析表头（关键修复：保留空列，只去掉首尾空元素）
        header_line = table_lines[0]
        # 分割表头：split后去掉首尾空元素，中间空列保留
        header_parts = header_line.split('|')
        # 去掉首尾空元素（Markdown表格通常以|开头/结尾）
        if not header_parts[0].strip():
            header_parts = header_parts[1:]
        if header_parts and not header_parts[-1].strip():
            header_parts = header_parts[:-1]
        # 表头列名做strip，但保留空列
        headers = [col.strip() for col in header_parts]
        logger.info(f"解析到表头：{headers}（共{len(headers)}列）")
        
        # 解析数据行（关键修复：保留空列，对齐列数）
        data = []
        for line_num, line in enumerate(table_lines[2:], start=3):
            if not line or '|' not in line:
                continue
                
            # 分割数据行：同样去掉首尾空元素，保留中间空列
            line_parts = line.split('|')
            if not line_parts[0].strip():
                line_parts = line_parts[1:]
            if line_parts and not line_parts[-1].strip():
                line_parts = line_parts[:-1]
            
            # 对每个列值做strip（空列会变成""）
            columns = [col.strip() for col in line_parts]
            
            # 对齐列数：不足补空字符串，多余截断
            if len(columns) < len(headers):
                columns += [''] * (len(headers) - len(columns))
            elif len(columns) > len(headers):
                columns = columns[:len(headers)]
            
            # 构造行字典
            row_dict = {headers[i]: columns[i] for i in range(len(headers))}
            data.append(row_dict)
        
        logger.info(f"解析到{len(data)}行有效表格数据")
        return data
    
    def extract_translation_pairs(self, table_data: List[Dict]) -> List[Dict]:
        """
        从表格数据中精准提取中英文翻译对
        
        Args:
            table_data: 解析后的表格数据
            
        Returns:
            翻译对列表
        """
        translation_pairs = []
        
        for row_idx, row in enumerate(table_data):
            # 1. 优先匹配固定列名（你的表格结构：英文术语/中文翻译）
            source_text = row.get("英文术语", "").strip()
            target_text = row.get("中文翻译", "").strip()
            
            # 过滤无效数据（英文/中文为空的行）
            if not source_text or not target_text:
                logger.debug(f"第{row_idx+1}行跳过：英文='{source_text}', 中文='{target_text}'")
                continue
            
            # 生成唯一ID（结合索引编号避免重复）
            index_id = row.get("索引编号", "")
            id_str = f"{index_id}_{source_text}_{target_text}".encode('utf-8')
            text_hash = hashlib.md5(id_str).hexdigest()[:16]
            
            # 构造翻译对（完善元数据）
            pair = {
                'id': text_hash,
                'source_text': source_text,
                'target_text': target_text,
                'source_lang': self.source_lang,
                'target_lang': self.target_lang,
                'metadata': {
                    'index_id': index_id,
                    'abbreviation': row.get("常用缩写", "").strip(),
                    'source_ext': row.get("来源&扩展", "").strip(),
                    'remarks': row.get("备注", "").strip(),
                    'original_row': row
                }
            }
            translation_pairs.append(pair)
        
        logger.info(f"提取到{len(translation_pairs)}个有效翻译对")
        return translation_pairs
    
    def read_markdown_file(self, file_path: str) -> str:
        """
        读取Markdown文件内容（增强错误处理）
        
        Args:
            file_path: 文件路径
            
        Returns:
            文件内容字符串
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            logger.info(f"成功读取文件：{file_path}（大小：{len(content)}字节）")
            return content
        except FileNotFoundError:
            logger.error(f"文件不存在：{file_path}")
        except PermissionError:
            logger.error(f"无权限读取文件：{file_path}")
        except Exception as e:
            logger.error(f"读取文件失败 {file_path}: {str(e)}")
        return ""
    
    def process_markdown_files(self, file_paths: List[str]) -> List[Dict]:
        """
        处理多个Markdown文件
        
        Args:
            file_paths: Markdown文件路径列表
            
        Returns:
            所有文件的翻译对合并列表
        """
        all_pairs = []
        
        for file_path in file_paths:
            # 读取文件
            content = self.read_markdown_file(file_path)
            if not content:
                continue
            
            # 解析表格
            table_data = self.parse_markdown_table(content)
            if not table_data:
                logger.warning(f"{file_path} 未解析到表格数据")
                continue
            
            # 提取翻译对
            pairs = self.extract_translation_pairs(table_data)
            logger.info(f"从 {file_path} 提取到 {len(pairs)} 个翻译对")
            
            # 添加文件来源信息
            for pair in pairs:
                pair['metadata']['source_file'] = Path(file_path).name
            
            all_pairs.extend(pairs)
        
        logger.info(f"所有文件处理完成，总计提取 {len(all_pairs)} 个翻译对")
        return all_pairs
    
    def save_to_csv(self, data: List[Dict], output_path: str):
        """
        保存为CSV文件（包含完整元数据）
        
        Args:
            data: 翻译对数据
            output_path: 输出文件路径
        """
        if not data:
            logger.warning("没有数据可保存到CSV")
            return
        
        # 准备CSV数据（展开元数据）
        csv_data = []
        for item in data:
            csv_row = {
                'id': item['id'],
                'index_id': item['metadata'].get('index_id', ''),
                'source_text': item['source_text'],
                'target_text': item['target_text'],
                'abbreviation': item['metadata'].get('abbreviation', ''),
                'remarks': item['metadata'].get('remarks', ''),
                'source_file': item['metadata'].get('source_file', '')
            }
            csv_data.append(csv_row)
        
        # 保存为CSV（utf-8-sig兼容Excel）
        df = pd.DataFrame(csv_data)
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        logger.info(f"CSV数据已保存到: {output_path}（{len(df)}行）")
    
    def save_to_json(self, data: List[Dict], output_path: str):
        """
        保存为JSON文件（Elasticsearch批量导入格式）
        
        Args:
            data: 翻译对数据
            output_path: 输出文件路径
        """
        if not data:
            logger.warning("没有数据可保存到JSON")
            return
        
        # Elasticsearch批量导入格式（NDJSON）
        es_data = []
        for item in data:
            # 索引操作
            index_op = {
                "index": {
                    "_index": "ai_terminology",  # 更贴合业务的索引名
                    "_id": item['id']
                }
            }
            # 文档数据（精简元数据，保留核心信息）
            doc_data = {
                "source_text": item['source_text'],
                "target_text": item['target_text'],
                "source_lang": item['source_lang'],
                "target_lang": item['target_lang'],
                "index_id": item['metadata'].get('index_id'),
                "abbreviation": item['metadata'].get('abbreviation'),
                "remarks": item['metadata'].get('remarks'),
                "source_file": item['metadata'].get('source_file')
            }
            
            es_data.append(index_op)
            es_data.append(doc_data)
        
        # 保存为NDJSON
        with open(output_path, 'w', encoding='utf-8') as f:
            for item in es_data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        
        logger.info(f"Elasticsearch批量数据已保存到: {output_path}（{len(es_data)//2}条文档）")
        logger.info(f"导入命令：curl -XPOST 'localhost:9200/_bulk' -H 'Content-Type: application/json' --data-binary @{output_path}")


# 使用示例
if __name__ == "__main__":
    # 创建转换器实例
    converter = MarkdownToElasticsearchConverter(source_lang='en', target_lang='zh')
    
    # 替换为你的Markdown文件路径
    file_paths = [
        "D:\\Code\\PythonProjects\\RagTranslationWorkflow\\AI for Science.md", 
        "D:\\Code\\PythonProjects\\RagTranslationWorkflow\\Machine Learning.md"
    ]
    
    # 处理文件
    results = converter.process_markdown_files(file_paths)
    
    # 保存结果
    if results:
        # 保存为CSV（便于查看和编辑）
        csv_output = "D:\\Code\\PythonProjects\\RagTranslationWorkflow\\translation_pairs.csv"
        converter.save_to_csv(results, csv_output)
        
        # 保存为Elasticsearch批量导入格式
        json_output = "D:\\Code\\PythonProjects\\RagTranslationWorkflow\\es_bulk_data.json"
        converter.save_to_json(results, json_output)
        
        # 显示前几条数据（验证结果）
        print("\n===== 前5条转换结果 =====")
        for i, item in enumerate(results[:5]):
            print(f"{i+1}. ID: {item['id']}")
            print(f"   索引编号: {item['metadata']['index_id']}")
            print(f"   英文术语: {item['source_text']}")
            print(f"   中文翻译: {item['target_text']}")
            print(f"   常用缩写: {item['metadata']['abbreviation']}")
            print()
    else:
        print("未提取到任何翻译对，请检查：")
        print("1. 文件路径是否正确")
        print("2. Markdown表格格式是否符合要求（包含|分隔符）")
        print("3. 表格是否有「英文术语」和「中文翻译」列")

INFO:__main__:成功读取文件：D:\Code\PythonProjects\RagTranslationWorkflow\AI for Science.md（大小：72861字节）
INFO:__main__:解析到表头：['索引编号', '英文术语', '中文翻译', '常用缩写', '来源&扩展', '备注']（共6列）
INFO:__main__:解析到491行有效表格数据
INFO:__main__:提取到491个有效翻译对
INFO:__main__:从 D:\Code\PythonProjects\RagTranslationWorkflow\AI for Science.md 提取到 491 个翻译对
INFO:__main__:成功读取文件：D:\Code\PythonProjects\RagTranslationWorkflow\Machine Learning.md（大小：48972字节）
INFO:__main__:解析到表头：['索引编号', '英文术语', '中文翻译', '常用缩写', '来源&扩展', '备注']（共6列）
INFO:__main__:解析到726行有效表格数据
INFO:__main__:提取到726个有效翻译对
INFO:__main__:从 D:\Code\PythonProjects\RagTranslationWorkflow\Machine Learning.md 提取到 726 个翻译对
INFO:__main__:所有文件处理完成，总计提取 1217 个翻译对
INFO:__main__:CSV数据已保存到: D:\Code\PythonProjects\RagTranslationWorkflow\translation_pairs.csv（1217行）
INFO:__main__:Elasticsearch批量数据已保存到: D:\Code\PythonProjects\RagTranslationWorkflow\es_bulk_data.json（1217条文档）
INFO:__main__:导入命令：curl -XPOST 'localhost:9200/_bulk' -H 'Content-Type: application/json' --data-binary @D:\Code


===== 前5条转换结果 =====
1. ID: 1b27a43fd0e1e8d5
   索引编号: AITD-00007
   英文术语: Accuracy
   中文翻译: 准确率
   常用缩写: 

2. ID: 398238dcbc1e45e1
   索引编号: AITD-00015
   英文术语: Activation Function
   中文翻译: 激活函数
   常用缩写: 

3. ID: c85b46dd91e0c691
   索引编号: AITD-00016
   英文术语: Active Learning
   中文翻译: 主动学习
   常用缩写: 

4. ID: e48fe510aa40da3a
   索引编号: AITD-00056
   英文术语: Area Under ROC Curve
   中文翻译: AUC（ROC曲线下方面积，度量分类模型好坏的标准）
   常用缩写: AUC

5. ID: ddb239a0a2a70673
   索引编号: AITD-00059
   英文术语: Artificial Intelligence
   中文翻译: 人工智能
   常用缩写: AI

